In [12]:
from src.data import DataLoaders
from src import utils
import pathlib
import pandas as pd
from src.models.ModelClasses import FireRiskModels, SmokeAlarmModels
from matplotlib import pyplot as plt
from pathlib import Path
import geopandas as geo
import folium

from datetime import date

import numpy as np

import fiona

import pyproj

In [13]:
#get output
out = pd.read_csv( utils.DATA['model-outputs'] /  'CombinedModels-FileStructureColumns-pop50-2022-05-29.csv' )
out['GEOID'] = out['GEOID'].str[2:]
out

,GEOID,state,NFIRS_start,NFIRS_end,years_fires_reported,total_fires_reported,max_fires_1k,median_fires_1k,total_fires_reported_sev,max_fires_1k_sev,...,race_pct_two+,age_pct_under25,age_pct_over65,disability_pct,snap_past_12_mo,single_unit_home,multi_unit_home,mobile_home,boat_van_rv_home,spanish_speaking
0,010010201001,AL,2011,2017,2,2.0,1.342282,1.342282,2.684564,1.342282,...,0.000000,0.363758,0.036242,0.239437,0.183099,0.820423,0.024648,0.154930,0.000000,0.102113
1,010010201002,AL,2011,2017,4,6.0,1.581028,1.185771,1.581028,0.790514,...,0.054545,0.348617,0.114625,0.278509,0.096491,0.925054,0.000000,0.074946,0.000000,0.046053
2,010010202001,AL,2011,2017,5,7.0,3.141361,1.047120,2.094241,1.047120,...,0.018750,0.372917,0.161458,0.458549,0.303109,0.768844,0.201005,0.030151,0.000000,0.000000
3,010010202002,AL,2011,2017,5,10.0,3.777148,0.944287,2.832861,1.888574,...,0.012945,0.303398,0.094660,0.280973,0.172566,0.821138,0.168699,0.010163,0.000000,0.011062
4,010010203001,AL,2011,2017,6,10.0,1.269036,0.634518,2.115059,0.423012,...,0.035956,0.302876,0.134095,0.379854,0.082524,0.995112,0.000000,0.004888,0.000000,0.037621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216022,560459511001,WY,2011,2017,1,2.0,1.372684,1.372684,1.372684,1.372684,...,0.049417,0.234043,0.175704,0.219941,0.054252,0.711035,0.053470,0.228669,0.006826,0.000000
216023,560459511002,WY,2011,2017,1,1.0,0.632111,0.632111,0.000000,NaN,...,0.004849,0.208513,0.215517,0.276139,0.010724,0.706250,0.000000,0.293750,0.000000,0.009383
216024,560459513001,WY,2011,2017,4,5.0,1.737619,0.868810,0.000000,NaN,...,0.000000,0.325804,0.076455,0.210325,0.172084,0.709910,0.054054,0.236036,0.000000,0.000000
216025,560459513002,WY,2011,2017,2,2.0,0.970874,0.970874,0.000000,NaN,...,0.017986,0.256295,0.341727,0.394973,0.075404,0.720325,0.160976,0.118699,0.000000,0.000000


In [14]:
#only keep some of the ACS columns
ACS_cols_to_use =['did_not_work_past_12_mo',
               'house_pct_live_alone',
               'pct_alt_heat',
               'house_yr_pct_before_1960',
               'inc_pct_poverty',
               'race_pct_black_or_amind']

ACS_data = out[['GEOID'] + ACS_cols_to_use]
ACS_data = ACS_data.set_index('GEOID')

ACS_data

,did_not_work_past_12_mo,house_pct_live_alone,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind
GEOID,,,,,,
010010201001,0.355019,0.221831,0.017606,0.221831,0.191946,0.214765
010010201002,0.309021,0.127193,0.032895,0.079229,0.044269,0.082213
010010202001,0.509589,0.344560,0.000000,0.135678,0.382199,0.591667
010010202002,0.309886,0.338496,0.019912,0.235772,0.111426,0.461974
010010203001,0.294054,0.276699,0.000000,0.030303,0.042301,0.227157
...,...,...,...,...,...,...
560459511001,0.301663,0.324047,0.366569,0.308305,0.094029,0.000000
560459511002,0.288010,0.258713,0.825737,0.197500,0.062579,0.014547
560459513001,0.232123,0.330784,0.105163,0.556757,0.261512,0.000000


In [15]:
#get SVI info
SVI =  DataLoaders.SVIData(ACS_data)
SVI.data


,inc_pct_poverty,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4
geos,,,,,
010010201001,0.191946,0.3913,0.5597,0.3735,0.0993
010010201002,0.044269,0.3913,0.5597,0.3735,0.0993
010010202001,0.382199,0.7065,0.3585,0.5821,0.7299
010010202002,0.111426,0.7065,0.3585,0.5821,0.7299
010010203001,0.042301,0.5659,0.8729,0.6429,0.3882
...,...,...,...,...,...
560459511001,0.094029,0.3925,0.6370,0.2331,0.8555
560459511002,0.062579,0.3925,0.6370,0.2331,0.8555
560459513001,0.261512,0.5093,0.6043,0.2862,0.6949


In [16]:
#merge main dataframe with SVI info
out = out.merge(SVI.data, how = 'left', left_on = 'GEOID', right_index = True, suffixes = ('', 'svi'))
out = out.round(3)

In [17]:
# ENTER STATE 2 number code here as string then run the following cells 
#STATE_2_NUM_CODE = '13' #Georgia
#STATE_2_NUM_CODE = '24' #Maryland
#STATE_2_NUM_CODE = '47' #Tennessee
#STATE_2_NUM_CODE = '51' #Virginia
#STATE_2_NUM_CODE = '25' #Massachusetts
#STATE_2_NUM_CODE = '20' #Kansas
#STATE_2_NUM_CODE = '46' #South Dakota
#STATE_2_NUM_CODE = '41' #Oregon
states = {}
states['Arizona'] = '04'
states['Oregon'] = '41'
states['Georgia'] = '13'
states['Maryland'] = '24'
states['Tennessee'] = '47'
states['Virginia'] = '51'
states['Massachusetts'] = '25'
states['Kansas'] = '20'
states['SouthDakota'] = '46'

#curr_state = 'Oregon'

In [18]:
def get_gdf(states, curr_state, out):
    #get geo info
    p = Path.cwd()
    shp_p =  p / 'Data' / 'Master Project Data'/ 'Tiger_censusBlocks_2016' / f'tl_2016_{states[curr_state]}_bg.zip'
    gdf = geo.read_file( 'zip://' + str(shp_p) ) 
    gdf = gdf.merge(out,on = 'GEOID')
    #only keep geoids with populations 50 or more
    #gdf = gdf[gdf['tot_population']>=50]
    
    return gdf

In [19]:
#all models with inputs
def allModelsMap(gdf, ACS_cols_to_use, curr_state):
    #get starting location and zoom level
    start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()
    m = folium.Map(location=start_loc, zoom_start=11 , tiles=None, zoom_control = False)
    
    #create base map
    base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
    folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
    base_map.add_to(m)
    
    #cz = folium.Control.zoom({position: 'bottomright'})
    #cz.add_to(m)

    #make label info
    toolTip = {}
    toolTip['CombinedRiskScore'] = ['GEOID', 'CombinedRiskScore','fire_propensity_risk_score','fire_severity_risk_score','smoke_detector_risk_score', 'RPL_THEME1','tot_population']
    toolTip['fire_risk_score'] = ['GEOID', 'fire_propensity_risk_score', 'fire_severity_risk_score', 'total_fires_reported','total_fires_reported_sev', 'max_fires_1k','max_fires_1k_sev', 'median_fires_1k','median_fires_1k_sev']+ACS_cols_to_use
    #toolTip['fire_propensity_risk_score'] = ['GEOID', 'fire_propensity_risk_score', 'total_fires_reported', 'max_fires_1k', 'median_fires_1k']+ACS_cols_to_use
    #toolTip['fire_severity_risk_score'] = ['GEOID', 'fire_severity_risk_score','total_fires_reported_sev', 'max_fires_1k_sev', 'median_fires_1k_sev']+ACS_cols_to_use
    toolTip['smoke_detector_risk_score'] = ['GEOID', 'smoke_detector_risk_score', 'RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4', 'geography','recorded_home_visits', 'inc_pct_poverty' ]
    toolTip['RPL_THEME1'] = ['GEOID', 'RPL_THEME1']
    
    #make layers
    #layers = ['CombinedRiskScore', 'fire_propensity_risk_score', 'fire_severity_risk_score','smoke_detector_risk_score', 'RPL_THEME1']
    layers = ['CombinedRiskScore', 'fire_risk_score','smoke_detector_risk_score', 'RPL_THEME1']
    for l in layers:
        print(l)
        # plot chorpleth over the base map
        cp = folium.Choropleth(gdf,                                # geo data
                      data=gdf,                           # data
                      key_on='feature.properties.GEOID', # feature.properties.key
                      columns=['GEOID', l],   # [key, value]
                      fill_color='RdPu',                     # cmap
                      line_weight=0.1,                       # line wight (of the border)
                      line_opacity=0.5,                      # line opacity (of the border)
                      overlay = False,
                      legend_name = l.replace('_',' '),
                      name =  l.replace('_',' ')
                      ).add_to(m)    # name on the legend color bar
        folium.GeoJsonPopup(toolTip[l]).add_to(cp.geojson)

    # add layer controls
    folium.LayerControl().add_to(m)
    #layer1.add_to(m)
    
    #save
    name = f"all_models_pop50_{curr_state}-{date.today()}.html"
    m.save( str(utils.DATA['map-outputs'] /  name))


In [20]:
#go through single state
#key = 'Arizona'
#gdf = get_gdf(states, key, out)
#allModelsMap(gdf, ACS_cols_to_use, key)

In [21]:
#Go through states and make map
for key in states.keys():
    print(key)
    gdf = get_gdf(states, key, out)
    allModelsMap(gdf, ACS_cols_to_use, key)

Arizona
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Oregon
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Georgia
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Maryland
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Tennessee
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Virginia
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Massachusetts
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
Kansas
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1
SouthDakota
CombinedRiskScore
fire_risk_score
smoke_detector_risk_score
RPL_THEME1


In [10]:
#compare old and new
out_old = pd.read_csv( utils.DATA['model-outputs'] /  'CombinedModels-FileStructureColumns-2022-04-29.csv' )
out_old['GEOID'] = out_old['GEOID'].str[2:]
out_old = out_old.set_index('GEOID')
out_old_models = out_old[['CombinedRiskScore']]
out_old_models['fire_risk_score'] = out_old.loc[:,['fire_propensity_risk_score', 'fire_severity_risk_score']].mean(axis = 1, skipna = True)

C:\Users\cle9a\AppData\Local\Temp\ipykernel_14992\2695222596.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_old_models['fire_risk_score'] = out_old.loc[:,['fire_propensity_risk_score', 'fire_severity_risk_score']].mean(axis = 1, skipna = True)


In [11]:
def get_gdf(states, curr_state, out, out_old_models):
    #get geo info
    p = Path.cwd()
    shp_p =  p / 'Data' / 'Master Project Data'/ 'Tiger_censusBlocks_2016' / f'tl_2016_{states[curr_state]}_bg.zip'
    gdf = geo.read_file( 'zip://' + str(shp_p) ) 
    gdf = gdf.merge(out,on = 'GEOID')
    gdf = gdf.merge(out_old_models, how = 'left',left_on = 'GEOID', right_index = True, suffixes = ('_new', '_old'))
    #only keep geoids with populations 50 or more
    gdf = gdf[gdf['tot_population']>=50]
    
    return gdf

In [12]:
#comparison
def allModelsMap(gdf, ACS_cols_to_use, curr_state):
    #get starting location and zoom level
    start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()
    m = folium.Map(location=start_loc, zoom_start=11 , tiles=None, zoom_control = False)
    
    #create base map
    base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
    folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
    base_map.add_to(m)
    
    #cz = folium.Control.zoom({position: 'bottomright'})
    #cz.add_to(m)

    #make label info
    toolTip = {}
    toolTip['CombinedRiskScore_new'] = ['GEOID', 'CombinedRiskScore_new','fire_risk_score_new','fire_propensity_risk_score','fire_severity_risk_score','smoke_detector_risk_score', 'RPL_THEME1','tot_population']
    toolTip['CombinedRiskScore_old'] = ['GEOID', 'CombinedRiskScore_old','fire_risk_score_old','fire_propensity_risk_score','fire_severity_risk_score','smoke_detector_risk_score', 'RPL_THEME1','tot_population']
    toolTip['fire_risk_score_new'] = ['GEOID', 'fire_risk_score_new', 'CombinedRiskScore_new','fire_propensity_risk_score','fire_severity_risk_score','fire_risk_score_new','fire_risk_score_old']
    toolTip['fire_risk_score_old'] = ['GEOID', 'fire_risk_score_old', 'CombinedRiskScore_old','fire_propensity_risk_score','fire_severity_risk_score','fire_risk_score_new','fire_risk_score_old']
    toolTip['fire_propensity_risk_score'] = ['GEOID', 'fire_propensity_risk_score', 'total_fires_reported', 'max_fires_1k', 'median_fires_1k']+ACS_cols_to_use
    toolTip['fire_severity_risk_score'] = ['GEOID', 'fire_severity_risk_score','total_fires_reported_sev', 'max_fires_1k_sev', 'median_fires_1k_sev']+ACS_cols_to_use
    #toolTip['smoke_detector_risk_score'] = ['GEOID', 'smoke_detector_risk_score', 'RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4', 'geography','recorded_home_visits', 'inc_pct_poverty' ]
    #toolTip['RPL_THEME1'] = ['GEOID', 'RPL_THEME1']
    
    #make layers
    layers = ['CombinedRiskScore_new', 'CombinedRiskScore_old', 'fire_risk_score_new', 'fire_risk_score_old', 'fire_propensity_risk_score', 'fire_severity_risk_score']
    for l in layers:
        print(l)
        # plot chorpleth over the base map
        cp = folium.Choropleth(gdf,                                # geo data
                      data=gdf,                           # data
                      key_on='feature.properties.GEOID', # feature.properties.key
                      columns=['GEOID', l],   # [key, value]
                      fill_color='RdPu',                     # cmap
                      line_weight=0.1,                       # line wight (of the border)
                      line_opacity=0.5,                      # line opacity (of the border)
                      overlay = False,
                      legend_name = l.replace('_',' '),
                      name =  l.replace('_',' ')
                      ).add_to(m)    # name on the legend color bar
        folium.GeoJsonPopup(toolTip[l]).add_to(cp.geojson)

    # add layer controls
    folium.LayerControl().add_to(m)
    #layer1.add_to(m)
    
    #save
    name = f"all_models_compare_{curr_state}-{date.today()}.html"
    m.save( str(utils.DATA['map-outputs'] /  name))


In [14]:
#Go through states and make map
for key in states.keys():
    print(key)
    gdf = get_gdf(states, key, out, out_old_models)
    allModelsMap(gdf, ACS_cols_to_use, key)

Maryland
CombinedRiskScore_new
CombinedRiskScore_old
fire_risk_score_new
fire_risk_score_old
fire_propensity_risk_score
fire_severity_risk_score
Virginia
CombinedRiskScore_new
CombinedRiskScore_old
fire_risk_score_new
fire_risk_score_old
fire_propensity_risk_score
fire_severity_risk_score
Massachusetts
CombinedRiskScore_new
CombinedRiskScore_old
fire_risk_score_new
fire_risk_score_old
fire_propensity_risk_score
fire_severity_risk_score


In [23]:
key = 'Arizona'
gdf = get_gdf(states, key, out, out_old_models)
allModelsMap(gdf, ACS_cols_to_use, key)

CombinedRiskScore_new
CombinedRiskScore_old
fire_risk_score_new
fire_risk_score_old
fire_propensity_risk_score
fire_severity_risk_score


In [ ]:
p = Path.cwd()

shp_p =  p / 'Data' / 'Master Project Data'/ 'Tiger_censusBlocks_2016' / f'tl_2016_{states[curr_state]}_bg.zip'

gdf = geo.read_file( 'zip://' + str(shp_p) ) 

#out['GEOID'] = out.index

gdf = gdf.merge(out,on = 'GEOID')

In [ ]:
gdf

In [ ]:
gdf = gdf[gdf['tot_population']>=50]
gdf

In [ ]:
#all models with inputs
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

toolTip = {}
toolTip['CombinedRiskScore'] = ['GEOID', 'CombinedRiskScore','fire_propensity_risk_score','fire_severity_risk_score','smoke_detector_risk_score', 'tot_population']
toolTip['fire_propensity_risk_score'] = ['GEOID', 'fire_propensity_risk_score', 'total_fires_reported', 'max_fires_1k', 'median_fires_1k']+ACS_cols_to_use
toolTip['fire_severity_risk_score'] = ['GEOID', 'fire_severity_risk_score','total_fires_reported_sev', 'max_fires_1k_sev', 'median_fires_1k_sev']+ACS_cols_to_use
toolTip['smoke_detector_risk_score'] = ['GEOID', 'smoke_detector_risk_score', 'RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4', 'geography','recorded_home_visits', 'inc_pct_poverty' ]

layers = ['CombinedRiskScore', 'fire_propensity_risk_score', 'fire_severity_risk_score','smoke_detector_risk_score']
for l in layers:
    print(l)
    # plot chorpleth over the base map
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', l],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = l.replace('_',' '),
                  name =  l.replace('_',' ')
                  ).add_to(m)    # name on the legend color bar
    folium.GeoJsonTooltip(toolTip[l]).add_to(cp.geojson)

# add layer controls
folium.LayerControl().add_to(m)
#layer1.add_to(m)

name = f"all_models{curr_state}-{date.today()}.html"
m.save( utils.DATA['model-outputs'] /  name)

In [ ]:
#all models with inputs
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

toolTip = {}
toolTip['fire_propensity_risk_score'] = ['GEOID', 'fire_propensity_risk_score', 'fire_propensity_risk_score_rank', 'Prop_Max', 'Prop_Median']+ACS_cols_to_use+['years_fire_records']
toolTip['fire_propensity_risk_score_rank'] = ['GEOID', 'fire_propensity_risk_score','fire_propensity_risk_score_rank', 'Prop_Max', 'Prop_Median']+ACS_cols_to_use+['years_fire_records']
toolTip['smoke_detector_risk_score'] = ['GEOID', 'smoke_detector_risk_score', 'smoke_detector_risk_frac', 'RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4', 'geography','detectors_predicted', 'inc_pct_poverty' ]
toolTip['smoke_detector_risk_frac'] = ['GEOID', 'smoke_detector_risk_score', 'smoke_detector_risk_frac', 'RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4', 'geography','detectors_predicted', 'inc_pct_poverty' ]

layers = ['fire_propensity_risk_score','fire_propensity_risk_score_rank','smoke_detector_risk_score','smoke_detector_risk_frac']
for l in layers:
    print(l)
    # plot chorpleth over the base map
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', l],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = l.replace('_',' '),
                  name =  l.replace('_',' ')
                  ).add_to(m)    # name on the legend color bar
    folium.GeoJsonTooltip(toolTip[l]).add_to(cp.geojson)

# add layer controls
folium.LayerControl().add_to(m)
#layer1.add_to(m)

name = f"maryland_original_rank_models-{date.today()}.html"
m.save(name)

In [ ]:
#overview map- each model
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

toolTip = ['GEOID', 'CombinedRiskScore','fire_propensity_risk_score','fire_severity_risk_score','smoke_detector_risk_score']

layers = ['CombinedRiskScore','fire_propensity_risk_score', 'fire_severity_risk_score', 'smoke_detector_risk_score' ]
for l in layers:
    # plot chorpleth over the base map
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', l],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = l.replace('_',' '),
                  name =  l.replace('_',' ')
                  ).add_to(m)    # name on the legend color bar
    folium.GeoJsonTooltip(toolTip).add_to(cp.geojson)

# add layer controls
folium.LayerControl().add_to(m)
#layer1.add_to(m)


m.save('georgia_overview.html')

In [ ]:
#smoke alarm and svi
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

# plot chorpleth over the base map
themes = ['RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4']
cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', 'smoke_detector_risk_score'],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = 'smoke_detector_risk_score',
                  name = 'smoke_detector_risk_score'
                  ).add_to(m)    # name on the legend color bar
 
folium.GeoJsonTooltip(['GEOID', 'smoke_detector_risk_score', 'geography','detectors_predicted']+themes).add_to(cp.geojson)

for t in themes:
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', t],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = t,
                  name = t
                  ).add_to(m)    # name on the legend color bar
 
    folium.GeoJsonTooltip(['GEOID']+['smoke_detector_risk_score']+themes).add_to(cp.geojson)
# add layer controls
folium.LayerControl().add_to(m)


m.save('georgia_smoke_svi.html')

In [ ]:
#inputs for propensity and severity models
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

toolTip = {}
toolTip['fire_propensity_risk_score'] = ['GEOID', 'fire_propensity_risk_score']+ACS_cols_to_use+['years_fire_records']
toolTip['fire_severity_risk_score'] = ['GEOID', 'fire_severity_risk_score']+ACS_cols_to_use+['years_fire_records']

layers = ['fire_propensity_risk_score', 'fire_severity_risk_score']
for l in layers:
    # plot chorpleth over the base map
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', l],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = l.replace('_',' '),
                  name =  l.replace('_',' ')
                  ).add_to(m)    # name on the legend color bar
    folium.GeoJsonTooltip(toolTip[l]).add_to(cp.geojson)

# add layer controls
folium.LayerControl().add_to(m)
#layer1.add_to(m)


m.save('georgia_fire_models.html')

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
geoid1 = out[out['GEOID'].str[:-1]=='13089023603']
geoid1

In [ ]:
geoid2 = out[out['GEOID']=='130890235043']
geoid2

In [ ]:
for i in range(134):
    print(gdf.columns[i])

In [ ]:
breakdown_smokealarm = pd.read_csv(r'.\Data\Model Outputs\SmokeAlarmPredictiveModel_USA_SVI_pov.csv')
breakdown_smokealarm['GEOID'] = breakdown_smokealarm['geoid'].str[2:]
breakdown_smokealarm

In [ ]:
gdf = gdf.merge(breakdown_smokealarm,on = 'GEOID')
gdf

In [ ]:
gdf['rd_SVI_pov'] = np.clip(gdf['rd_SVI_pov'],0,100)  
gdf['rd_SVI_pov'] = 100-gdf['rd_SVI_pov']
gdf['Just_SVI_pov'] = np.clip(gdf['Just_SVI_pov'],0,100)  
gdf['Just_SVI_pov'] = 100-gdf['Just_SVI_pov']

In [ ]:
gdf

In [ ]:
#smoke alarm and svi
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

# plot chorpleth over the base map
themes = ['RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4']
layers = ['rd_SVI_pov', 'Just_SVI_pov']
cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', 'smoke_detector_risk_score'],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = 'smoke_detector_risk_score',
                  name = 'smoke_detector_risk_score'
                  ).add_to(m)    # name on the legend color bar
 
folium.GeoJsonTooltip(['GEOID', 'smoke_detector_risk_score', 'geography_x','detectors_predicted']+layers+themes).add_to(cp.geojson)

for t in layers:
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', t],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = t,
                  name = t
                  ).add_to(m)    # name on the legend color bar
 
    folium.GeoJsonTooltip(['GEOID','smoke_detector_risk_score', 'geography_x','detectors_predicted']+layers+themes).add_to(cp.geojson)
# add layer controls
folium.LayerControl().add_to(m)


m.save('maryland_smoke_svi_test.html')